<i><b>Public AI</b></i>
<br>
###  &nbsp;&nbsp; **✎&nbsp;&nbsp;Week 2. Convolution Neural Network**
# Section 6. Fully Convolutional Network Segmentation_4


In [ ]:
# If You use in Colab, You Should run this script
import os
if (not os.path.exists("./multicampus_segmentation_generator") and
    not "multicampus_segmentation_generator" in os.getcwd()):
    !git clone https://github.com/public-ai/multicampus_segmentation_generator.git
os.chdir("./multicampus_segmentation_generator")

In [ ]:
from tensorflow.keras.layers import Layer, Conv2D, MaxPool2D, Conv2DTranspose, ZeroPadding2D, Concatenate
from tensorflow.keras.layers import Input, UpSampling2D, BatchNormalization, Flatten, Dense, Dropout
from tensorflow.keras.layers import Softmax, Add
from tensorflow.keras.layers import Lambda, concatenate
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import optimizers
import tensorflow.keras.backend as K 
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt 
from keras.utils import np_utils
import numpy as np
import tensorflow as tf
import cv2
import sys 
sys.path.append('../')
from generator import fasion_mnist_generator

# Load Dataset

In [ ]:
(train_images, train_labels), (test_images, test_labels) = fasion_mnist_generator(1000, 10)
print('Train Image shape : ', train_images.shape)
print('Train Labels shape : ', train_labels.shape)
print('Test Image shape : ', test_images.shape)
print('Test Labels shape : ', test_labels.shape)

In [ ]:
fig, axes = plt.subplots(1,2)
fig.set_size_inches((10,10))
sample_image = train_images[0]
axes[0].imshow(train_images[0])
axes[0].set_title('Image')

sample_mask = np.argmax(train_labels[0], axis=-1)
axes[1].imshow(sample_mask)
axes[1].set_title('Label')

plt.show()

# Build Model Architecture 

![Imgur](https://i.imgur.com/Dp86kkd.png)

In [ ]:
tf.reset_default_graph()

# 위 그림에 만족하는 Convolution Neural Network를 구성하고 모델을 생성해주세요
# fix me!

In [ ]:
def mean_iou(y_true, y_pred):
    
    n_classes = 11 
    y_cls = tf.argmax(y_true, axis=-1)
    pred_cls = tf.argmax(y_pred, axis=-1)

    y_uniques = tf.to_int64(tf.unique(tf.reshape(y_cls, [-1]))[0])
    pred_uniques = tf.to_int64(tf.unique(tf.reshape(pred_cls, [-1]))[0])
    
    uniques = tf.to_int64(tf.unique(tf.concat([y_uniques, pred_uniques], axis=0))[0])

    def get_union(index):
        union = tf.reduce_sum(tf.to_int64(tf.equal(y_cls, index) | tf.equal(pred_cls, index)))
        return union

    def get_intersection(index):
        inter = tf.reduce_sum(tf.to_int64(tf.equal(y_cls, index) & tf.equal(pred_cls, index)))
        return inter 
    
    unions = tf.map_fn(get_union, uniques)
    inters = tf.map_fn(get_intersection, uniques)
    return tf.reduce_mean(inters / unions)

def batch_mean_iou(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float64)
    y_pred = tf.cast(y_pred, tf.float64)
    n_classes = 11 
    stacked_cls = tf.stack([y_pred, y_true], axis=-1)
    print(stacked_cls)
    
    
    bm_iou = tf.reduce_mean(tf.map_fn(lambda cls: mean_iou(cls[..., 0], cls[..., 1]), stacked_cls))
    return bm_iou

In [ ]:
# compile & fitting 
# metric 으로 [batch_mean_iou]를 사용해주세요
# optimizer 로는 Nesterov Accelerated Gradient(NAG)를 사용해주세요
# learning_rate=0.005, decay=1e-6, momentum=0.9
# fix me!

In [ ]:
pred = model.predict(test_images[:100]/255.)
pred_cls = np.argmax(pred, axis=-1)

In [ ]:
fig, axes = plt.subplots(2,10)
ognl_axes = np.array(axes)[0, :].flatten()
pred_axes = np.array(axes)[1, :].flatten()
fig.set_size_inches(30,10)
for i in range(10):
    ognl_axes[i].imshow(test_images[i])
    pred_axes[i].imshow(pred_cls[i] == 0)

---
⊙ Copyright(c) 2020 by PublicAI. All rights reserved <br>
All pictures, codes, writings cannot be copied without permission. <br>
Writen by PAI(info@publicai.co.kr) <br>
last updated on 2020/01/4 <br>

---